In [3]:
import numpy as np

## Класс для работы с числами в GF(16)

In [4]:
"""
Следующие функции переписывают таблицу мультипликативной группы поля из 16 элементов
https://ru.wikipedia.org/wiki/%D0%9A%D0%BE%D0%BD%D0%B5%D1%87%D0%BD%D0%BE%D0%B5_%D0%BF%D0%BE%D0%BB%D0%B5
"""
def get_step(num):
    g_s = np.asarray([-1000, 0, 1, 4, 2, 8, 5, 10, 3, 14, 9, 7, 6, 13, 11, 12, 1])
    return g_s[num]
def get_num(step):
    if (step == -1000):
        return 0
    g_n = np.asarray([1, 2, 4, 8, 3, 6, 12, 11, 5, 10, 7, 14, 15, 13, 9, 1])
    return g_n[step]

In [5]:
class GF(object):
    """
    Класс, дающий возможность вычисления в поле GF(2^4)
    Здесь переопределны операции сложения, вычитания, умножения, деления
    """
    def __init__(self, msg):
        self.polyn = np.zeros((4), dtype = np.int32)
        self.step = -1000
        if msg != 0:
            self.step = get_step(msg)
        
        for i in range(4):
            self.polyn[i] = msg % 2 
            #a^0 = 0,0,0,1
            msg = msg // 2
    def __add__(self, other):
        new_polyn = (self.polyn + other.polyn) % 2
        new_num = new_polyn[0] + 2 * new_polyn[1] + 4 * new_polyn[2] + 8 * new_polyn[3]
        return GF(new_num)
    
    def __sub__(self, other):
        return ((self + other))
    
    def __mul__(self, other):
        if (self.step == -1000 or other.step == -1000):
            return GF(0)
        new_step = (self.step + other.step) % 15
        return GF(get_num(new_step))
    def __truediv__(self, other):
        if (self.step == -1000 or other.step == -1000):
            return GF(0)
        new_step = (self.step - other.step + 15) % 15
        return GF(get_num(new_step))
        

In [6]:
get_num((GF(13)*GF(15)).step)

7

# Кодирование

#### Функция деления полиномов в поле GF(16)

In [7]:
#5*x**3 + 4*x**2 + 1 -> [1, 0, 4, 5]
def normalize(poly):
    while poly and poly[-1].step == -1000:
        poly.pop()
    if poly == []:
        poly.append(GF(0))

def poly_divmod(p, q):
    p, q = p[:], q[:]
    normalize(p)
    normalize(q)
    if len(p) < len(q):
        return p
    sh = len(p) - len(q)
    q = [GF(0)] * sh + q
    div = q[-1]
    for i in range(sh + 1):
        mult = p[-1] / div
        if mult != 0:
            d = [mult * u for u in q]
            p = [u - v for u, v in zip(p, d)]

        p.pop()
        q.pop(0)
    normalize(p)
    return p



#### Функция умножения в поле GF(16)

In [8]:
def mul(p, q, d):
    """
    Умножение полиномов в поле GF(16)
    """
    res = [GF(0)] * (d)
    normalize(p)
    normalize(q)
    for i in range(len(p)):
        for j in range(len(q)):
            if (i + j < d):
                res[i + j] += p[i] * q[j]
    return res

#### Функция для декодирования из объекта класса в число


In [9]:
def dec(a):
    """
    Функция для декодирования из объекта класса в число
    """
    return get_num(a.step)

#### Функция для конструирования g(x)

In [10]:
def get_G(d):
    """
    Функция для конструирования по данным n и k g(x)=(x+a)(x+a^2)...(x+a^d-1)
    """
    g = [GF(0)] * (d)
    g[0] = GF(2)
    g[1] = GF(1)
    x = GF(2)
    for i in range(d - 1 - 1):
        cur = [GF(0)] * (d)
        x = x * GF(2)
        cur[0] = x
        cur[1] = GF(1)
        g = mul(g, cur, d)
    return g

### Непосредственно функция кодирования 

In [11]:
def count_Reed_Solomon(msg, n, k):
    p = [0] * (n - k) + msg 
    d = n - k + 1
    
    g = get_G(d)
    q = poly_divmod(list(map(GF, p)), g)
    
    q.reverse()
    msg.reverse()
    p = msg + [0] * (n - k)
    q = [0] * (k) + list(map(dec, q))
    C = np.asarray(p) + np.asarray(q)
    return C [::-1]

### Пример исользования
#### count_Reed_Solomon([message], N, K)
Полезное сообщение 7, 5, 10, 0, 9, 1, 1, 1, 9

Избыточная информация 3, 15, 15, 14, 6, 13 

In [12]:
count_Reed_Solomon([7,5,10,0,9,1,1,1,9], 15,9)

array([ 3, 15, 15, 14,  6, 13,  7,  5, 10,  0,  9,  1,  1,  1,  9])

# Декодирование

#### Функция для подстановки в многочлен polynom значение a (все в поле GF(16))

In [13]:
def count_polx (polynom, a):
    """
    Функция для подстановки в многочлен polynom значение a
    """
    x = GF(1)
    result = GF(0)
    for i in range(len(polynom)):
            result += x * polynom[i]
            x *= a
    return result

#### Функции для работы с матрицами в поле GF(16)

In [14]:
def mul_matrix_GF(A, B, t):
    """
    Умножение квадратной матрицы на столбец
    """
    M = [GF(0)] *(t)
    for i in range(t):
        for k in range(t):
            M[i] += GF(A[i][k]) * GF(B[k])
    M1 = [0] * (t)
    for i in range(len(M)):
        M1[i] = get_num((M[i]).step)
    return M1
def get_M(s, t):
    """
    Получение матрицы, необходимой для декодирования
    """
    M = [[0 for x in range(t)] for y in range(t)]
    V = [0] * t
    for i in range(t):
        for j in range(t):
            M[i][j] = s[t  - 1 + i - j]
        V[i] = s[t + i]
    return M, V
def get_minor(M,i,j):
    """
    Получение минора матрицы без i строчки, j столбца
    """
    return [row[ : j] + row[j + 1 : ] for row in (M[ : i] + M[i + 1 : ])]
def get_det_Gf(M):
    """
    Получение дискреминанта в поле GF(16)
    """
    if len(M) == 2:
        return GF(M[0][0]) * GF(M[1][1]) + GF(M[0][1]) * GF(M[1][0])
    det = GF(0)
    for c in range(len(M)):
        det += GF(M[0][c]) * get_det_Gf(get_minor(M, 0, c))
    return det

In [15]:
def M_T(M):
    """
    Получение транспонированной матрицы
    """
    for i in range(len(M)):
        for j in range(i, len(M[0])):
            M[i][j], M[j][i] = M[j][i], M[i][j]
    return M
def get_M1(M):
    """
    Получение обратной матрицы
    """
    det = get_det_Gf(M)
    if det.step == -1:
        return []
    if len(M) == 2:
        return [[get_num((GF(M[1][1]) / det).step), get_num((GF(M[0][1]) / det).step)],
                [get_num((GF(M[1][0]) / det).step), get_num((GF(M[0][0]) / det).step)]]
    C = []
    for r in range(len(M)):
        rows = []
        for c in range(len(M)):
            m = get_minor(M,r,c)
            rows.append(get_num((get_det_Gf(m) / det).step))
        C.append(rows)
    C = M_T(C)
    return C

In [96]:
def decode_Reed_Solomon(msg, n, k):
    d = n - k + 1
    g = get_G(d)
    #1 вычислить msg(x) mod g(x)
    m = list(map(GF, msg))
    q = poly_divmod(m, g)
    normalize(q)
    if list(map(dec,q)) == [0]:
        #2 если остаток 0, то просто декодировать
        return msg[(n - k) : (n)]
       
    #3 найти полином S: si = q(a^(i+1)), где q = msg(x) mod g(x)
    x = GF(2)
    s = []
    for i in range(n - k):
        s = s + [count_polx(q, x)]
        x = x * (GF(2))
#     print(list(map(dec,s)))
    
    #4 построить матрицу M
    t = (n - k) // 2
    while t > 0:
        M, V = get_M(list(map(dec,s)), t)
        det = get_num(get_det_Gf(M).step)
        if det != 0:
            break
        t -= 1
    M1 = get_M1(M)
    #с помощью M вычилить многочлен L
    L = mul_matrix_GF(M1, V, t)
    L = [1] + L
#     print(L)
    #5 вычислить производную L 
    L1 = [0] * len(L)
    for i in range(len(L) - 1):
        if i % 2 == 0:
            L1[i] = L[i + 1]
    #6
    W = mul(list(map(GF,L)), s, n - k)
#     print(L1, L, list(map(dec,s)), list(map(dec,W)))
    
    #7 подобрать корни L
    x = GF(1)
    Need_to_add = [0] * n
    for i in range(16):
        res = count_polx(list(map(GF,L)), x)
        if (res.step == -1000):
            x1 = GF(1) / x
            y = count_polx(W, x) / count_polx(list(map(GF,L1)), x)
            print(get_num(y.step), x1.step)
            Need_to_add[x1.step] = get_num(y.step)
        x *= GF(2)
    msg = np.asarray(msg)
    Need_to_add = np.asarray(Need_to_add)
    for i in range(len(msg)):
        msg[i] = get_num((GF(msg[i]) + GF(Need_to_add[i])).step)
    return msg[n-k:]
    

In [97]:
decode_Reed_Solomon([ 3, 15, 15, 14, 6, 13, 7, 5, 10, 0, 9, 1, 1, 1, 9], 15,9)

[7, 5, 10, 0, 9, 1, 1, 1, 9]

In [102]:
decode_Reed_Solomon([ 3, 0, 15, 14, 6, 13, 7, 5, 10, 0, 9, 1, 0, 1, 9], 15,9)

1 12
15 1


array([ 7,  5, 10,  0,  9,  1,  1,  1,  9])

In [143]:
def deleted_Reed_Solomon(msg, n, k, st):
    d = n - k + 1
    g = get_G(d)
    #1 вычислить msg(x) mod g(x)
    m = list(map(GF, msg))
    q = poly_divmod(m, g)
    normalize(q)
    3,
       
    #3 найти полином S: si = q(a^(i+1)), где q = msg(x) mod g(x)
    x = GF(2)
    s = []
    for i in range(n - k):
        s = s + [count_polx(q, x)]
        x = x * (GF(2))
#     print(list(map(dec,s)))
    
    L = [GF(0)] * d
    L[0] = GF(1) / GF(get_num(st[0]))
    L[1] = GF(1)
    for i in range(1, len(st)):
        cur = [GF(0)] * (d)
        cur[0] = GF(1) / GF(get_num(st[i]))
        cur[1] = GF(1)
        L = mul(L, cur, d)
    L = list(map(dec,L))
#     print(L)
    L1 = [0] * len(L)
    for i in range(len(L) - 1):
        if i % 2 == 0:
            L1[i] = L[i + 1]
    #6
    W = mul(list(map(GF,L)), s, n - k)

    #7 подобрать корни L
    Need_to_add = [0] * n
    for i in range(len(st)):
        #print((count_polx(list(map(GF,L)), GF(1) / GF(st[i]))).step)
        x = GF(1) / GF(get_num(st[i]))
        x1 = GF(get_num(st[i]))
        
        y = count_polx(W, x) / count_polx(list(map(GF,L1)), x)
#         print(get_num(y.step), x1.step)      
        Need_to_add[x1.step] = get_num(y.step)
        
    msg = np.asarray(msg)
    Need_to_add = np.asarray(Need_to_add)
    for i in range(len(msg)):
        msg[i] = get_num((GF(msg[i]) + GF(Need_to_add[i])).step)
    return msg[n-k:]

In [144]:
deleted_Reed_Solomon([ 3, 15, 0, 14, 0, 13, 7, 5, 10, 0, 9, 1, 0, 1, 9], 15,9, [2,4,12])

array([ 7,  5, 10,  0,  9,  1,  1,  1,  9])